# Branch InsertNameHere
function: `experimentx`

In [1]:
install.packages(c("grf", "doParallel", "doSNOW", "doRNG", "FNN","mvtnorm"))

library("mvtnorm")

library(grf)
library(doParallel)
library(doSNOW)
library(doRNG)
library(FNN)

# Part 1: Data Generating Processes (Design 1 to 4)
# basic functions

nonlinear <- function(x) {
  1 + 1 / (1 + exp(-20 * (x - 1/3)))
}
propensity <- function(x) {
  0.5 + dbeta(x,1,2) / 8
}

# Design 1: Strong Treatment Effect Heterogeneity (lineaer and nonlinear)

design1 <- function(n,
                    n.test,
                    d,
                    prop,
                    noise,
                    form) {
  X <- matrix(runif(n * d, 0, 1), n, d)
  X.test = matrix(runif(n.test * d, 0, 1), n.test, d)
  W <- rbinom(n, 1, prop)
  
  if (form == "linear") {
    Y <- (2*X[,1] + X[,2]) * W + X[,2] + X[,3] + rnorm(n, 0, noise)
    tau <- 2*X.test[,1] + X.test[,2]
  }
  
  if (form == "nonlinear") {
    Y <- sapply(X[,1], nonlinear) * sapply(X[,2], nonlinear) * W + sapply(X[,2], nonlinear) * sapply(X[,3], nonlinear) + rnorm(n, 0, noise)
    tau <- sapply(X.test[,1], nonlinear) * sapply(X.test[,2], nonlinear)
    
  }
  return(list(X=X,
              X.test=X.test,
              W=W,
              Y=Y,
              tau=tau))
}

experimenta1 <- function(n, 
                         n.test, 
                         d, 
                         prop,
                         noise)
{
  mean <- 2
  X <- matrix(rnorm(n * d, mean, 1), n, d)
  X.test = matrix(rnorm(n.test * d, mean, 1), n.test, d)
  W <- rbinom(n, 1, prop)
  
  tau <- 4
  Y <- X[,1] + X[,2] + (4* W)  + rnorm(n, 0, noise)
  
  return(list(X=X,
              X.test=X.test,
              W=W,
              Y=Y,
              tau=tau))
}

experimenta2 <- function(n, 
                         n.test, 
                         d, 
                         prop,
                         noise)
{
  mean <- 2
  X <- matrix(rnorm(n * d, mean, 1), n, d)
  X.test <- matrix(rnorm(n.test * d, mean, 1), n.test, d)
  W <- rbinom(n, 1, prop)
  
  tau <- (2*X.test[,1]) + X.test[,2]
  Y <- ((2*X[,1]) + X[,2])* W + X[,3] + rnorm(n, 0, noise)
  
  return(list(X=X,
              X.test=X.test,
              W=W,
              Y=Y,
              tau=tau))
}

experimenta3 <- function(n, 
                         n.test, 
                         d, 
                         prop,
                         noise)
{
  mean <- 2
  X <- matrix(rnorm(n * d, mean, 1),n,d)
  X.test = matrix(rnorm(n.test * d, mean, 1), n.test, d)
  W <- rbinom(n, size = 1, p = sapply(X[,1], propensity))
  
  tau <- (2*X.test[,1]) + X.test[,2]
  Y <- (2*X[,1] + X[,2]) * W + X[,3] + rnorm(n, 0, noise)
  
  return(list(X=X,
              X.test=X.test,
              W=W,
              Y=Y,
              tau=tau))
}

cor_ar <- function(n, rho) 
{
  exponent <- abs(matrix(1:n - 1, nrow = n, ncol = n, byrow = TRUE) - 
                    (1:n - 1))^2
  
  pre_round <- rho^exponent
  # pre_round[abs(pre_round) < 0.5] <- 00.0000000001
  pre_round
}

experimentb <- function(n, 
                        n.test, 
                        d, 
                        prop,
                        noise)
{
  noise <- 0.1
  mu_0 <- rep(1,d)
  mu_0.test <- rep(1,d)
  corcov <- cor_ar(d, 0.9)
  corcov.t <- t(corcov)
  semi_def <- corcov %*% corcov.t
  X <- rmvnorm(n, mean = mu_0, sigma = semi_def)
  X.test <- rmvnorm(n.test, mean = mu_0.test, sigma = semi_def)
  
  W <- rbinom(n, 1, prop)
  
  tau <- 2*X.test[,1] + X.test[,2]
  Y <- (2*X[,1] + X[,2]) * W + 0.5* X[,3] + rnorm(n, 0, noise)
  
  return(list(X=X,
              X.test=X.test,
              W=W,
              Y=Y,
              tau = tau))
}

experimentc1 <- function(n, 
                        n.test, 
                        d, 
                        prop,
                        noise)
{
  n <- 4000
  n.test <- 1000
  d <- 0.1
  prop <- 0.5
  noise <- 0.5
  mean <- rep(1,6)
  # use d=10 as a fixed value so that the dimensions can be used for sigma
  X <- matrix(rnorm(n * 6, mean, 1), n, 6)
  X.test = matrix(rnorm(n.test * 6, mean, 1), n.test, 6)
  W <- rbinom(n, 1, prop)
  
  # 100 for 100 subgroups, d as the sigma for the normal distribution of the function u(s)
  U <- rnorm(500, 0, d)
  U.r <- rep(U,n/500)
  S <- rep(seq(1,500,1), n/500)
  X <- cbind(X,S,U.r)
  colnames(X)[colnames(X) == 'U.r'] <- 'U'
  
  S.test <- rep(seq(1,500,1), n.test/500)
  U.test <- rep(U,n.test/500)
  
  X.test <- cbind(X.test,S.test,U.test)
  colnames(X)[colnames(X) == 'U.test'] <- 'U'
  colnames(X)[colnames(X) == 'S.test'] <- 'S'

  tau <- abs(2*X.test[,1]) + X.test[,2] + U.test
  Y <- tau * W + 0.5*X[,3] + U.r + rnorm(n, 0, noise) 

  return(list(X=X,
              X.test=X.test,
              W=W,
              Y=Y,
              tau=tau))
}

# Part 2: Performance Evaluation

mse <- function(predictions, true) {
  return(mean((predictions-true)^2))
}



bias <- function(predictions, true){
  return(abs(mean(predictions-true)))
}

covered <- function(predictions, true, sigma) {
  return(mean(abs(predictions-true) / sigma <= 1.96))
}

evaluate <- function(predictions, true, sigma) {
  return(list(mse = mse(predictions,true),
              bias = bias(predictions,true),
              coverage = covered(predictions,true,sigma)))
}

# Part 3: Estimators

# Causal Forest with user-specific parameters
# Honesty can be disabled by setting 'honest = FALSE'
# Local centering can be disabled by setting Y.hat and W.hat to vectors of zeros

CF_estimator <- function(X,
                         X.test,
                         Y,
                         W,
                         Y.hat = NULL,
                         W.hat = NULL,
                         num.trees = 1000,
                         clusters = NULL,
                         honesty = TRUE,
                         honesty.fraction = 0.5,
                         honesty.prune.leaves = TRUE,
                         min.node.size = 5) {
  CF <- causal_forest(X,
                      Y,
                      W,
                      Y.hat = Y.hat,
                      W.hat = W.hat,
                      num.trees = num.trees,
                      clusters = clusters,
                      min.node.size = min.node.size,
                      honesty = honesty,
                      honesty.fraction = honesty.fraction,
                      honesty.prune.leaves = honesty.prune.leaves,
                      seed = 1)
  
  estimates <- predict(CF,
                       X.test,
                       estimate.variance = TRUE)
  ate <- average_treatment_effect(CF, target.sample = "all")
  
  return(list(predictions = estimates$predictions,
              sigma = sqrt(estimates$variance.estimates),
              ate = ate,
              var.imp = variable_importance(CF)))
}

# KNN

NN_estimate <- function(X,
                        X.test,
                        Y,
                        W,
                        k) {
  
  knn.control <- knn.reg(X[W==0,], X.test, Y[W==0], k = k)$pred # outcome of control units
  knn.treated <- knn.reg(X[W==1,], X.test, Y[W==1], k = k)$pred # outcome of treated units
  
  # treatment effect
  knn.tau <- knn.treated - knn.control
  
  # estimating the variance
  knn.control2 <- knn.reg(X[W==0,], X.test, Y[W==0]^2, k = k)$pred
  knn.treated2 <- knn.reg(X[W==1,], X.test, Y[W==1]^2, k = k)$pred
  knn.control.var <- (knn.control2 - knn.control^2) / (k - 1)
  knn.treated.var <- (knn.treated2 - knn.treated^2) / (k - 1)
  
  return(list(predictions = knn.tau,
              sigma = sqrt(knn.control.var + knn.treated.var),
              ate = mean(knn.tau)))
}

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘DiceKriging’, ‘lmtest’, ‘sandwich’, ‘RcppEigen’, ‘foreach’, ‘iterators’, ‘snow’, ‘rngtools’


Loading required package: foreach

Loading required package: iterators

Loading required package: parallel

Loading required package: snow


Attaching package: ‘snow’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, clusterSplit, makeCluster, parApply,
    parCapply, parLapply, parRapply, parSapply, splitIndices,
    stopCluster


Loading required package: rngtools



In [2]:
simulation_procedure <- function(d, form) {
  n <- 4000
  n.test <- 1000
  noise <- 0.5
  data <- experimentc1(n,          # The name of this function needs to be changed.
                       n.test,
                       d=d,
                       prop = 0.5,
                       noise)
  
  # causal forest  
  cf <- CF_estimator(data$X,
                     data$X.test,
                     data$Y,
                     data$W,
                     num.trees = 1000)
  
  cf.evaluation <- evaluate(cf$predictions, data$tau, cf$sigma)
    
  return (data.frame(n = n,
             d = d,
             cf.mse = cf.evaluation$mse, 
             cf.bias = cf.evaluation$bias,
             cf.sigma = mean(cf$sigma),
             cf.coverage = cf.evaluation$coverage))
  
}

In [ ]:
n.simulation <- 1000
parameter.values <- c(6,8,12,16,20)
form <- "linear"
columns = c("n",
            "d",
            "cf.mse",
            "cf.bias",
            "cf.sigma",
            "cf.coverage")


cores=detectCores()
cl <- makeCluster(cores[1])
registerDoSNOW(cl)
  
  # progess bar
pb <- txtProgressBar(max=n.simulation, style=3)
progress <- function(n) 
{
  setTxtProgressBar(pb, n)
}
opts <- list(progress=progress)
  
  # initialize dataframe
output <- setNames(data.frame(matrix(ncol = length(columns), nrow = 0)),
                     columns)
  
  # run simulation
  # using the doRNG package to guarantee reproducible results
set.seed(1)

for(parameter in parameter.values){
  print(paste("Running ",parameter, ":"))
  results = foreach(i=1:n.simulation,
                    .combine=rbind,
                    .options.snow=opts,
                    .packages=c('grf', 'FNN')) %dopar% {
                      simulation_procedure(parameter, form)
                    }
  output <- rbind.data.frame(output, colMeans(results))
}
output <- setNames(output, columns)

  |                                                                      |   0%[1] "Running  6 :"
  |=                                                                     |   2%

In [ ]:
print(output)